# Videos Information2 [update, description]

(1) 실시간 스트리밍경우 소리만 나오는듯 하다(=아닌 경우도 존재하는 듯). 날짜로 스트리밍인지 아닌지 구분 가능.  
(2) 진짜 더보기가 없는 경우  
(3) \<a href의 경우 찾음 1238번째  
(4) 언어 바꿔서 크롤링하기 - 완성  
(5) 동영상이 사용자에 의해 삭제된 상황 처리 39번째 혹은 비공개 동영상인 경우. - 완성  
(6) 더보기가 안 들어간 것도 아니고 이상하게 한두글자 들어간 경우 - 완성  

In [1]:
import pandas as pd
from selenium import webdriver
from bs4 import BeautifulSoup
from slacker import Slacker
import time
from datetime import datetime 
import traceback

## Using slack Bot

In [22]:
f = open("../../slack_token_check_bot.txt")
token_check = f.readline()[:-1]
slack_check = Slacker(token_check)
f.close()
token_check[-3:]

'Vyf'

In [23]:
f = open("../../slack_token_noti_bot.txt")
token_noti = f.readline()[:-1]
slack_noti = Slacker(token_noti)
f.close()
token_noti[-3:]

'Pmo'

## Read / Make CSV file

In [4]:
csv_name = "youtuber_videos_info2.csv"

In [5]:
rm_f = ""
try: 
    videos_df = pd.read_csv(csv_name, encoding = "utf-8")
    rm_f = "Read file"
    print("Read file")
except:
    rm_f = "Exception handling"
    print("Exception handling")
    temp_df = pd.read_csv("youtuber_videos_info.csv", encoding = "utf-8")
    videos_df = temp_df
    videos_df["update"] = None
    videos_df["description"] = None
    videos_df["thumbnail"] = None
    videos_df.to_csv(csv_name, encoding = "utf-8", index = False)
    videos_df = pd.read_csv(csv_name, encoding = "utf-8")
    videos_df = videos_df.fillna(0)
    videos_df.to_csv(csv_name, encoding = "utf-8", index = False)
    videos_df = pd.read_csv(csv_name, encoding = "utf-8")

Read file


In [6]:
type(videos_df["update"][4079])

str

In [7]:
videos_df.head(3)

,youtuber_name,video_id,update,description,thumbnail
0,씬님,https://www.youtube.com/watch?v=UTXuSv8e18E,게시일: 2017. 5. 29.,♥ 안녕하세요 여러분 씬님입니다! 오늘은 꽁지님과 함께 콜라보를 하게 되었어요! 왼...,https://i.ytimg.com/vi/UTXuSv8e18E/maxresdefau...
1,씬님,https://www.youtube.com/watch?v=_BsClDZgRaY,게시일: 2017. 5. 22.,♥ 안녕하세요 여러분 씬님입니다. \n저번에 소개팅 메이크업에 이어 오늘은 시집가는...,https://i.ytimg.com/vi/_BsClDZgRaY/maxresdefau...
2,씬님,https://www.youtube.com/watch?v=u_aOZxEiIFs,게시일: 2017. 5. 16.,♥ 안녕하세요 여러분 씬님입니다! 얼마전 투쿨포스쿨과 함께 진행한 팬미팅에 다녀왔는...,https://i.ytimg.com/vi/u_aOZxEiIFs/maxresdefau...


### Edit video_id

In [8]:
youtube = "https://www.youtube.com"
thumb_pre = "https://i.ytimg.com/vi/"
thumb_des = "/maxresdefault.jpg"
cnt_videos = len(videos_df)

In [9]:
start_time = datetime.now()

cnt_change_df = 0
for i in range(cnt_videos):
    if videos_df["video_id"][i][:19] != "https://www.youtube":
        cnt_change_df += 1
        temp_ad = youtube + videos_df["video_id"][i]
        videos_df["video_id"][i] = temp_ad
        videos_df.to_csv(csv_name, encoding = "utf-8", index = False)
print(datetime.now()-start_time)
print("Change of number : ", cnt_change_df)

0:00:00.033953
Change of number :  0


In [10]:
restart_for = videos_df

# Function

In [11]:
def get_address_bs(ad):
    open_address = driver.get(ad)
    get_source = driver.page_source
    global address_bs
    address_bs = BeautifulSoup(get_source, "html.parser")
    return address_bs

In [12]:
def private_delete_video(num, ad): #동영상을 볼 수 없습니다. #동영상이 사용자에 의해 삭제되었습니다.
    pride_bs = get_address_bs(ad)  #Content Warning      #removed by the user
    result = str(pride_bs.find_all("h1", {"class" : "message"}))
    while(1):
        if "\n" not in result:
            if "  " not in result:
                break
        result = result.replace("\n", "")
        result = result.replace("  ", "")
    result = result[46:-6]
    if "삭제되었습니다." in result:
        videos_df["update"][num] = "remove"
        videos_df["description"][num] = "remove"
        videos_df["thumbnail"][num] = "remove"
        slack_check.chat.post_message("#output", " *This video removd by user.*\n```Number : " + str(i) + ", "+ videos_df["video_id"] + "```", as_user=True)
    elif "볼 수 없습니다." in result:
        videos_df["update"][num] = "private"
        videos_df["description"][num] = "private"
        videos_df["thumbnail"][num] = "private"
        slack_check.chat.post_message("#output", " *Content Warning*\n```Number : " + str(i) + ", "+ videos_df["video_id"] + "```", as_user=True)

In [13]:
def find_upload_date(ad_bs):
    upload = ad_bs.find_all("div", {"id" : "watch-uploader-info"})
    upload = str(upload)
    upload = upload[63:-16]
    return upload

In [14]:
def find_description(ad_bs):
    global description
    describe = ad_bs.find_all("p", {"id" : "eow-description"})
    describe = str(describe)
    describe = describe[34:-5]
    describe = describe.replace("</a>", "")
    describe = describe.replace("<br/>", "\n")
    describe = describe.split("<a class=\"")
    for i in range(len(describe)):
        if "yt-uix-servicelink" in describe[i]:
            result = describe[i]
            result = result.split("target=\"_blank\">")
            #result = result.split("\">")
            describe[i] = result[1]
        if "yt-uix-sessionlink" in describe[i]:
            result = describe[i]
            result = result.split("\">")
            describe[i] = result[1]
    description = "".join(describe)
    return description

In [15]:
def set_korean():
    open_address = driver.get("https://www.youtube.com/watch?v=TGstCQ5qUeE")
    time.sleep(10)
    get_source = driver.page_source
    set_bs = BeautifulSoup(get_source, "html.parser")
    driver.find_element_by_id('yt-picker-language-button').click()
    time.sleep(5)
    driver.find_element_by_xpath("//*[@title='Korean']").click()

In [16]:
def set_result_ms():
    now = str(datetime.now())[:-7]
    now = datetime.strptime(str(now), '%Y-%m-%d %H:%M:%S').strftime('%s')
    result_ms = result_ms = "```Find Information : " + str(find_info) + "\nShort Description : " + str(short_ds) + "```" + "⏱ Time required : " + str(datetime.now()-start_time)
    result = dict()
    result["fallback"] = "Finish in Crawling"
    result["color"] = "danger"
    result["title"] = "Result For Video Information"
    result["text"] = result_ms
    result["footer"] = "Beauty"
    result["footer_icon"] = "http://cfile218.uf.daum.net/image/220C804A593401F80F7867"
    result["ts"] = now
    result['mrkdwn_in'] = ["text"]
    global result_att
    result_att = [result]

In [17]:
def set_error_ms(e):
    now = str(datetime.now())[:-7]
    now = datetime.strptime(str(now), '%Y-%m-%d %H:%M:%S').strftime('%s')
    error = dict()
    error["fallback"] = "⚠️ Error in Crawling"
    error["color"] = "warning"
    error["title"] = "⚠️ Error about Video Information"
    error["text"] = e
    error["footer"] = "Beauty"
    error["footer_icon"] = "http://cfile218.uf.daum.net/image/220C804A593401F80F7867"
    error["ts"] = now
    error['mrkdwn_in'] = ["text"]
    global error_att
    error_att = [error]

In [18]:
def set_estimated_ms(t, o):
    now = str(datetime.now())[:-7]
    now = datetime.strptime(str(now), '%Y-%m-%d %H:%M:%S').strftime('%s')
    esti = dict()
    esti["fallback"] = "🃏 Estimated time"
    esti["color"] = "#439FE0"
    esti["title"] = "🃏 Estimated time of completion"
    esti["text"] = "Estimated time : " + str(t) + "\nOne iterate : " + str(o)
    esti["footer"] = "Beauty"
    esti["footer_icon"] = "http://cfile218.uf.daum.net/image/220C804A593401F80F7867"
    esti["ts"] = now
    esti['mrkdwn_in'] = ["text"]
    global estimated_att
    estimated_att = [esti]

# Main

확인할 것  
(1) 비공개/삭제 동영상인 경우  
(2) Description 5자 미만 확인

In [19]:
videos_df.head(3)

,youtuber_name,video_id,update,description,thumbnail
0,씬님,https://www.youtube.com/watch?v=UTXuSv8e18E,게시일: 2017. 5. 29.,♥ 안녕하세요 여러분 씬님입니다! 오늘은 꽁지님과 함께 콜라보를 하게 되었어요! 왼...,https://i.ytimg.com/vi/UTXuSv8e18E/maxresdefau...
1,씬님,https://www.youtube.com/watch?v=_BsClDZgRaY,게시일: 2017. 5. 22.,♥ 안녕하세요 여러분 씬님입니다. \n저번에 소개팅 메이크업에 이어 오늘은 시집가는...,https://i.ytimg.com/vi/_BsClDZgRaY/maxresdefau...
2,씬님,https://www.youtube.com/watch?v=u_aOZxEiIFs,게시일: 2017. 5. 16.,♥ 안녕하세요 여러분 씬님입니다! 얼마전 투쿨포스쿨과 함께 진행한 팬미팅에 다녀왔는...,https://i.ytimg.com/vi/u_aOZxEiIFs/maxresdefau...


In [24]:
driver = webdriver.Firefox()
set_korean()

In [25]:
if rm_f != "Read file":
    slack_check.chat.post_message("#Process", "> * Start Crawling *", as_user = True)
    slack_check.chat.post_message("#output", "> *Find Description of less than 5 characters*", as_user = True)

short_ds = 0
find_info = 0

try:
    for i in range(cnt_videos):
        if i == 0:
            for_start_time = datetime.now()
        address = videos_df["video_id"][i]
        private_delete_video(i, address)
        if videos_df["update"][i] == 0:
            find_info += 1
            print(i)
            address_bs = get_address_bs(address)
            upload = find_upload_date(address_bs)
            description = find_description(address_bs)
            if len(description) < 5:
                short_ds += 1
                slack_check.chat.post_message("#output", "`Number : " + str(i) + "` "+ videos_df["video_id"][i] + "\n" + str(description), as_user=True)
            videos_df["update"][i] = upload
            videos_df["description"][i] = description
            thumbnail = videos_df["video_id"][i][32:]
            thumbnail = thumb_pre + thumbnail + thumb_des
            videos_df["thumbnail"][i] = thumbnail
            videos_df.to_csv(csv_name, encoding="utf-8", index=False)
        slack_check.chat.post_message("#process", "`Number : " + str(i) + "`\n" + str(videos_df["description"][i]), as_user = True)
        if i == 0:
            for_finish_time = datetime.now()
            one_for = for_finish_time - for_start_time
            total_for = one_for * cnt_videos
            estimated_time = str(for_start_time + total_for)
            print(estimated_time, one_for)
            set_estimated_ms(estimated_time, one_for)
            slack_noti.chat.post_message("#result", text = None, attachments = estimated_att, as_user=True)
    driver.quit()
    set_result_ms()
    slack_noti.chat.post_message("#result", text = None, attachments = result_att, as_user=True)

except:
    import sys
    import traceback
    e = sys.exc_info()
    rep = str(e[1])
    rep = rep.replace("'", "`")
    e = "[" + str(e[0])[8:-2] + "] " + rep
    traceB = str(traceback.format_exc())
    e = e + traceB
    set_error_ms(e)
    slack_noti.chat.post_message("#result", attachments = error_att, as_user=True)

2017-06-06 03:28:44.572922 0:00:02.167465


In [58]:
import traceback

In [59]:
try:
    dd
except:
    a = sys.last_traceback
    b = traceback.format_exc()

In [62]:
print(a)
b

'Traceback (most recent call last):\n  File "<ipython-input-59-b77658b58aee>", line 2, in <module>\n    dd\nNameError: name \'dd\' is not defined\n'